# Ejercicio de BYTE-PAIR ENCODING
Integrantes:
*   Aguilar Valenzuela Luis Hector
*   Camargo Loaiza Julio Andres
*   Minjares Neriz Victor Manuel

# Importaciones

In [4]:
import re, collections

# Funciones

### get_vocab(filename)
Función que recibe el nombre de el archivo de texto y devuelve un vocabulario de palabras con la frecuencia de cada palabra y un separador en cada palabra

In [10]:
def get_vocab(filename):
    # La funcion defaultdict crea un dictionario vacio. 
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            # La función strip quita los espacios al principio y al final de un string
            # La función split separa las palabras y las devuelve en un array
            words = line.strip().split()
            
            # Recorre cada palabra del arreglo de palabras
            for word in words:
                # Aqui se llena el diccionario. Agregara al diccionario el elemnto : un espacio +
                # la palabra + el simbolo de fin de palabra </w>.
                # NOTA : list() aqui puede ser opcional, tal vez.
                vocab[' '.join(list(word)) + ' </w>'] += 1 
    return vocab

get_vocab('miniCorpus.txt')

defaultdict(int,
            {'Y o , </w>': 1,
             'J u a n </w>': 1,
             'G a l l o </w>': 1,
             'd e </w>': 8,
             'A n d r a d a , </w>': 1,
             'e s c r i b a n o </w>': 1,
             'C á m a r a </w>': 1,
             'd e l </w>': 4,
             'R e y </w>': 1,
             'n u e s t r o </w>': 1,
             's e ñ o r , </w>': 1,
             'l o s </w>': 2,
             'q u e </w>': 6,
             'r e s i d e n </w>': 1,
             'e n </w>': 4,
             's u </w>': 1,
             'C o n s e j o , </w>': 1,
             'c e r t i f i c o </w>': 1,
             'y </w>': 10,
             'd o y </w>': 1,
             'f e </w>': 1,
             'q u e , </w>': 1,
             'h a b i e n d o </w>': 1,
             'v i s t o </w>': 1,
             'p o r </w>': 2,
             's e ñ o r e s </w>': 1,
             'd é l </w>': 1,
             'u n </w>': 1,
             'l i b r o </w>': 3,
             'i n t 

### get_tokens(vocab)

Funcion que recibe un diccionario de palabras, despues transforma el diccionario de palabras a uno de letras.

In [16]:
def get_tokens(vocab):

    # Declara un diccionario vacío
    tokens = collections.defaultdict(int)

    # Iteramos por cada palabra y tomando su respectiva frecuencia
    for word, freq in vocab.items():
        # Separa las palabras por letra
        word_tokens = word.split()
        print(word_tokens)
        # Llenamos el diccionario tokens con cada letra y su respectiva frecuencia
        for token in word_tokens:
            tokens[token] += freq
    return tokens

get_tokens(get_vocab('miniCorpus.txt'))

['Y', 'o', ',', '</w>']
['J', 'u', 'a', 'n', '</w>']
['G', 'a', 'l', 'l', 'o', '</w>']
['d', 'e', '</w>']
['A', 'n', 'd', 'r', 'a', 'd', 'a', ',', '</w>']
['e', 's', 'c', 'r', 'i', 'b', 'a', 'n', 'o', '</w>']
['C', 'á', 'm', 'a', 'r', 'a', '</w>']
['d', 'e', 'l', '</w>']
['R', 'e', 'y', '</w>']
['n', 'u', 'e', 's', 't', 'r', 'o', '</w>']
['s', 'e', 'ñ', 'o', 'r', ',', '</w>']
['l', 'o', 's', '</w>']
['q', 'u', 'e', '</w>']
['r', 'e', 's', 'i', 'd', 'e', 'n', '</w>']
['e', 'n', '</w>']
['s', 'u', '</w>']
['C', 'o', 'n', 's', 'e', 'j', 'o', ',', '</w>']
['c', 'e', 'r', 't', 'i', 'f', 'i', 'c', 'o', '</w>']
['y', '</w>']
['d', 'o', 'y', '</w>']
['f', 'e', '</w>']
['q', 'u', 'e', ',', '</w>']
['h', 'a', 'b', 'i', 'e', 'n', 'd', 'o', '</w>']
['v', 'i', 's', 't', 'o', '</w>']
['p', 'o', 'r', '</w>']
['s', 'e', 'ñ', 'o', 'r', 'e', 's', '</w>']
['d', 'é', 'l', '</w>']
['u', 'n', '</w>']
['l', 'i', 'b', 'r', 'o', '</w>']
['i', 'n', 't', 'i', 't', 'u', 'l', 'a', 'd', 'o', '</w>']
['E', 'l', '</w

defaultdict(int,
            {'Y': 2,
             'o': 53,
             ',': 14,
             '</w>': 139,
             'J': 1,
             'u': 18,
             'a': 56,
             'n': 39,
             'G': 1,
             'l': 36,
             'd': 44,
             'e': 90,
             'A': 1,
             'r': 34,
             's': 37,
             'c': 22,
             'i': 42,
             'b': 7,
             'C': 3,
             'á': 1,
             'm': 11,
             'R': 1,
             'y': 12,
             't': 23,
             'ñ': 3,
             'q': 7,
             'j': 1,
             'f': 2,
             'h': 9,
             'v': 10,
             'p': 17,
             'é': 1,
             'E': 1,
             'g': 6,
             'M': 2,
             'S': 1,
             'í': 3,
             ';': 2,
             '.': 2,
             'V': 1})

### get_stats(vocab)
Función que recibe un vocabulario (un diccionario con la frecuencia de cada palabra) y devuelve un diccionario con la frecuencia de los bigramas (pares de palabras consecutivos) en el vocabulario.

In [24]:
def get_stats(vocab):
    # Declara un diccionario vacío
    pairs = collections.defaultdict(int)

    # Iteramos por cada palabra y tomando su respectiva frecuencia
    for word, freq in vocab.items():
        # Separa las palabras por letra
        symbols = word.split()
        print(symbols)
        # Esto se lee "Recorre el largo de"
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

get_stats(get_vocab('miniCorpus.txt'))

['Y', 'o', ',', '</w>']
['J', 'u', 'a', 'n', '</w>']
['G', 'a', 'l', 'l', 'o', '</w>']
['d', 'e', '</w>']
['A', 'n', 'd', 'r', 'a', 'd', 'a', ',', '</w>']
['e', 's', 'c', 'r', 'i', 'b', 'a', 'n', 'o', '</w>']
['C', 'á', 'm', 'a', 'r', 'a', '</w>']
['d', 'e', 'l', '</w>']
['R', 'e', 'y', '</w>']
['n', 'u', 'e', 's', 't', 'r', 'o', '</w>']
['s', 'e', 'ñ', 'o', 'r', ',', '</w>']
['l', 'o', 's', '</w>']
['q', 'u', 'e', '</w>']
['r', 'e', 's', 'i', 'd', 'e', 'n', '</w>']
['e', 'n', '</w>']
['s', 'u', '</w>']
['C', 'o', 'n', 's', 'e', 'j', 'o', ',', '</w>']
['c', 'e', 'r', 't', 'i', 'f', 'i', 'c', 'o', '</w>']
['y', '</w>']
['d', 'o', 'y', '</w>']
['f', 'e', '</w>']
['q', 'u', 'e', ',', '</w>']
['h', 'a', 'b', 'i', 'e', 'n', 'd', 'o', '</w>']
['v', 'i', 's', 't', 'o', '</w>']
['p', 'o', 'r', '</w>']
['s', 'e', 'ñ', 'o', 'r', 'e', 's', '</w>']
['d', 'é', 'l', '</w>']
['u', 'n', '</w>']
['l', 'i', 'b', 'r', 'o', '</w>']
['i', 'n', 't', 'i', 't', 'u', 'l', 'a', 'd', 'o', '</w>']
['E', 'l', '</w

defaultdict(int,
            {('Y', 'o'): 1,
             ('o', ','): 4,
             (',', '</w>'): 14,
             ('J', 'u'): 1,
             ('u', 'a'): 3,
             ('a', 'n'): 5,
             ('n', '</w>'): 11,
             ('G', 'a'): 1,
             ('a', 'l'): 6,
             ('l', 'l'): 4,
             ('l', 'o'): 4,
             ('o', '</w>'): 24,
             ('d', 'e'): 18,
             ('e', '</w>'): 24,
             ('A', 'n'): 1,
             ('n', 'd'): 6,
             ('d', 'r'): 2,
             ('r', 'a'): 7,
             ('a', 'd'): 4,
             ('d', 'a'): 6,
             ('a', ','): 3,
             ('e', 's'): 12,
             ('s', 'c'): 2,
             ('c', 'r'): 1,
             ('r', 'i'): 2,
             ('i', 'b'): 5,
             ('b', 'a'): 1,
             ('n', 'o'): 3,
             ('C', 'á'): 1,
             ('á', 'm'): 1,
             ('m', 'a'): 4,
             ('a', 'r'): 7,
             ('a', '</w>'): 19,
             ('e', 'l'): 10,
        

### merge_vocab(pair, v_in)
Función que recibe una pareja de palabras (pair) y un vocabulario (v_in) y devuelve otro vocabulario nuevo (v_out) con las nuevas parejas de palabras concatenadas.

In [ ]:
def merge_vocab(pair, v_in):
    #Crea un diccionario vacío
    v_out = {}
    # Quita los caracteres especiales
    bigram = re.escape(' '.join(pair))
    # Crea expresión regular
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    # Reemplaza la cadena bigram con la pareja concatenada y lo agrega en v_out
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    # Regresa el nuevo vocabulario
    return v_out

# Codigo principal

In [ ]:
# Minicorpus a usar :
# https://drive.google.com/file/d/17h_rLrWL2xg3jD0U1CCseeaAd6t17yc0/view?usp=share_linket 

vocab = get_vocab('miniCorpus.txt')

print('==========')
print('Tokens Before BPE')
tokens = get_tokens(vocab)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))
print('==========')

num_merges = 15
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    tokens = get_tokens(vocab)
    print('Tokens: {}'.format(tokens))
    print('Number of tokens: {}'.format(len(tokens)))
    print('==========')